# Predicted Orbit Files

Predicted Orbit Files are generated from FOS for the three Swarm spacecrafts.

They are available on Swarm FTP at `/Auxiliary/Sat_x/MPLxORBPRE` for `mission` users.

## Parse predicted orbit file from ZIP archive

In [1]:
from pathlib import Path
from xml.etree import ElementTree
from zipfile import ZipFile

from dateutil.parser import parse

In [2]:
# path to the predicted orbit ZIP file
file = 'SW_OPER_MPLAORBPRE_20201126T000000_20201231T000000_2560.ZIP'

In [3]:
file = Path(file)

with ZipFile(file) as zf:
    et = ElementTree.parse(zf.open(file.with_suffix('.EEF').name))

# parse orbit state vectors
ns = {'': 'http://eop-cfi.esa.int/CFI'}
osvs = [
    {
        'TAI': parse(osv.findtext('TAI', namespaces=ns)[4:]),
        'UTC': parse(osv.findtext('UTC', namespaces=ns)[4:]),
        'UT1': parse(osv.findtext('UT1', namespaces=ns)[4:]),
        'Absolute_Orbit': int(osv.findtext('Absolute_Orbit', namespaces=ns)),
        'X': float(osv.findtext('X', namespaces=ns)),
        'Y': float(osv.findtext('Y', namespaces=ns)),
        'Z': float(osv.findtext('Z', namespaces=ns)),
        'VX': float(osv.findtext('VX', namespaces=ns)),
        'VY': float(osv.findtext('VY', namespaces=ns)),
        'VZ': float(osv.findtext('VZ', namespaces=ns))
    }
    for osv in et.iterfind('./Data_Block/List_of_OSVs/OSV', ns)
]

osvs

[{'TAI': datetime.datetime(2020, 11, 26, 0, 33, 15, 566936),
  'UTC': datetime.datetime(2020, 11, 26, 0, 32, 38, 566936),
  'UT1': datetime.datetime(2020, 11, 26, 0, 32, 38, 387528),
  'Absolute_Orbit': 39418,
  'X': 6810128.732,
  'Y': -200683.127,
  'Z': 4296.648,
  'VX': -20.1919,
  'VY': -144.666866,
  'VZ': 7642.501918},
 {'TAI': datetime.datetime(2020, 11, 26, 2, 6, 31, 920126),
  'UTC': datetime.datetime(2020, 11, 26, 2, 5, 54, 920126),
  'UT1': datetime.datetime(2020, 11, 26, 2, 5, 54, 740707),
  'Absolute_Orbit': 39419,
  'X': 6170133.04,
  'Y': -2889349.329,
  'Z': 4298.438,
  'VX': -76.051564,
  'VY': -124.783002,
  'VZ': 7642.425501},
 {'TAI': datetime.datetime(2020, 11, 26, 3, 39, 48, 230336),
  'UTC': datetime.datetime(2020, 11, 26, 3, 39, 11, 230336),
  'UT1': datetime.datetime(2020, 11, 26, 3, 39, 11, 50905),
  'Absolute_Orbit': 39420,
  'X': 4514786.355,
  'Y': -5102725.57,
  'Z': 4301.718,
  'VX': -119.317375,
  'VY': -84.322037,
  'VZ': 7642.211325},
 {'TAI': datetim